In [ ]:
USE {
    dependencies {
        implementation("org.jetbrains.kotlinx:dataframe:0.13.1")
    }
}


In [ ]:
import org.jetbrains.kotlinx.dataframe.*
import org.jetbrains.kotlinx.dataframe.api.*
import org.jetbrains.kotlinx.dataframe.io.*

import java.io.ByteArrayInputStream
import java.io.ByteArrayOutputStream


In [ ]:
val perl = ProcessBuilder(listOf("stat.pl", "--xlsx", "transactions.xlsx")).start()

val outputStream = ByteArrayOutputStream()
perl.inputStream.copyTo(outputStream)
perl.waitFor()
println(perl.exitValue())

val stat = DataFrame.readCSV(ByteArrayInputStream(outputStream.toByteArray()), parserOptions = ParserOptions(nullStrings = setOf("-")))
stat.writeExcel(File("stat.xlsx"), sheetName = "data", keepFile = true)
stat
